<h1>Using <i style="color:red;">SQLAlchemy</i>'s ORM in <i style="color:green;">Django</i></h1>

<h3><i>by Jack Camier, Python Developer</i></h3>

# Django is one of the most popular web frameworks in Python. As of this talk, 2027 companies reportedly use Django in their tech stacks, including YouTube, Bitbucket, Dropbox, Instagram, Pinterest, Udemy just to name a few...
https://stackshare.io/django

In [ ]:
%%HTML
<img src="django_logo.png" width="300"/>

https://www.djangoproject.com/

## Personally, I am a big fan and use it often. However, one aspect of Django that I don't like that much is its ORM. I am not alone and have heard from developers their frustrations with the ORM. It works great for simple projects and queries but whenever you try to do something more complex, you can run into problems.

## In this quick tutorial, I will show you a way you can use SQLAlchemy's ORM in conjunction with your Django Projects.

## We will use the Django tutorial as our baseline to see how this can work.
https://docs.djangoproject.com/en/3.0/intro/tutorial01/

## I am using the latest Django version as of this writing 3.0.5

In [ ]:
import django

django.VERSION

## Steps from the online tutorial, I had problems doing using the Jupyter Notebook. So these are the steps I did to get the project setup. django `mysite` is available in the git repo.

## created a virtualenv called `py37` with python 3.7.7

## You could do the same with `conda create --name py37 python=3.7`

## `django-admin startproject mysite`

## Go into the `mysite` directory and `python manage.py startapp polls`

## Create a URLconf in the `polls` directory, create a file called `urls.py`

## Next step is to point the root URLconf at the polls.urls module. In `mysite/urls.py`, add an import for django.urls.include and insert an include() in the urlpatterns list

## `python manage.py migrate`

## The migrate command looks at the INSTALLED_APPS setting and creates any necessary database tables according to the database settings in your `mysite/settings.py`

## `auth_user` is the key table that is created to manage users

## Create the models (Tables) in the `polls/models.py` module

## Add `'polls.apps.PollsConfig',` to the `mysite/settings.py` module. This is to allow Django to know that our app is in our project.

## The format is normally app_name.apps.App_nameConfig.

## So `polls` becomes `polls.apps.PollsConfig`

## It should look something like this now:

## Then, `python manage.py makemigrations polls` to update the migration

## Next, `python manage.py migrate` to build (create) the tables stored in migrations

# -------
## Up to this point, this has all been standard Django functionality. Now, this is where I will show a way you can use SQLAlchemy to working with Django

# -------

## Make sure you have SQLAlchemy installed
`pip install SQLAlchemy`
## I have included it in my requirements.txt file already in case you want to `pip install -r requirements.txt`

## First I created a module called `session.py` in the `polls` directory, hence polls/session.py

## These are all my imports:

## The first thing we need access to is the SQLite database that was created by Django.

## Let's cd into the path that contains `session.py ` 

In [1]:
## To see our current directory
import os

CURRENT_DIR = os.path.abspath('') # Use '' for notebooks but __file__ for .py files

CURRENT_DIR

'/Users/jacquescamier/JupyterProjects/dfw_sqlalchemy_talk'

In [2]:
session_dir = os.path.join(CURRENT_DIR, "mysite", "polls")

session_dir

os.chdir(session_dir)

## `BASE_DIR` is a global variable assigned in the `mysite/mysite/settings.py` file.


## We will use the same logic to access the SQLite database in the `session.py` module

In [3]:
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("session.py")))

BASE_DIR

'/Users/jacquescamier/JupyterProjects/dfw_sqlalchemy_talk/mysite'

## As you can see the BASE_DIR get us to the project directory of `mysite` which also contains the SQLite database file `db.sqlite3` 

In [4]:
os.listdir(BASE_DIR)

['mysite', 'db.sqlite3', 'polls', 'manage.py']

## To get the SQLAlchemy SQLite file path protocol

In [5]:
sqlite_path = os.path.join(BASE_DIR, 'db.sqlite3')
sqlite_path_str = f"sqlite:///{sqlite_path}"

sqlite_path_str

'sqlite:////Users/jacquescamier/JupyterProjects/dfw_sqlalchemy_talk/mysite/db.sqlite3'

## As mentioned in my earlier tutorial:
https://github.com/jcamier/dfw_sqlalchemy_talk/blob/master/intro_sqlalchemy.ipynb

## We will need to create the engine object

In [6]:
import sqlalchemy

engine = sqlalchemy.create_engine(sqlite_path_str)

## Create a session object to use SQLAlchemy's ORM and querying the tables

In [7]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

## Now one approach is to rewrite the classes using SQLAlchemy's (SA) syntax of our models so we can query them later with the ORM but this would not be efficient

## SA has a great tool called reflection that allows us access the models (Tables) already created. We just need to make a connection between them and the `metadata` object

In [8]:
from sqlalchemy.engine import reflection

metadata = sqlalchemy.MetaData()

metadata.reflect(bind=engine)

insp = reflection.Inspector.from_engine(engine)

## To create an object that SA can use by adding it to the `metadata` object, you would do something like this:
metadata.tables[`table_name`]

In [9]:
# Create table reflection

auth_user_table = metadata.tables['auth_user']
polls_choice_table = metadata.tables['polls_choice']
polls_question_table = metadata.tables['polls_question']

## Now we can see all the tables using this simple helper function I wrote

In [11]:
from typing import List

def get_all_tables() -> List:
    """
    `get_all_tables` helper function to get a list of all tables in the database
    
    :return list all table names: 
    """
    return insp.get_table_names()

django_tables = get_all_tables()

django_tables

['auth_group',
 'auth_group_permissions',
 'auth_permission',
 'auth_user',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'django_content_type',
 'django_migrations',
 'django_session',
 'polls_choice',
 'polls_question',
 'sqlite_sequence']

## And if you forgot the column names, here is another helper function I wrote. Honestly, in most case you would just go to the models.py to see them, but this is more to display the functionaliy of SA

## Be sure to use the metadata reflection table name assigned, not Django's. For example, it would be `auth_user_table` from above not `auth_user` because we are using SA and not the built-in Django ORM

In [13]:
def print_table_meta_columns(table) -> None:
    """
    `print_table_meta_columns helper function to stdout to terminal the column names of a table

    :param table:
    :return None:
    """

    print(table._columns)
    
print_table_meta_columns(table=auth_user_table)

['auth_user.id', 'auth_user.password', 'auth_user.last_login', 'auth_user.is_superuser', 'auth_user.username', 'auth_user.first_name', 'auth_user.email', 'auth_user.is_staff', 'auth_user.is_active', 'auth_user.date_joined', 'auth_user.last_name']


## At this point, we haven't really seen anything that would want us to switch to using SA's ORM vs Django's

## Let's write some test data to our database so we can play around with the ORM's functionality

In [10]:
import csv
import datetime as dt

In [53]:
django_users_file = os.path.join(CURRENT_DIR, 'django_users_data.csv')

with open(django_users_file, mode='r') as users_file:
    reader = csv.reader(users_file, delimiter=',')
    headers = next(reader, None)
    users_import_list = [{headers[0]: row[0], headers[1]: row[1], headers[2]: row[2], headers[3]: row[3],
                          headers[4]: row[4], headers[5]: int(row[5]), headers[6]: int(row[6]),
                          headers[7]: int(row[7]), "date_joined": dt.datetime.now()
                         } for idx, row in enumerate(reader)]
    

users_import_list

[{'username': 'admin@example.com',
  'password': 'test1234',
  'email': 'admin@example.com',
  'first_name': 'Admin',
  'last_name': '',
  'is_superuser': 1,
  'is_staff': 1,
  'is_active': 1,
  'date_joined': datetime.datetime(2020, 4, 23, 21, 39, 41, 507769)},
 {'username': 'chris@example.com',
  'password': 'test1234',
  'email': 'chris@example.com',
  'first_name': 'Chris',
  'last_name': 'Smith',
  'is_superuser': 0,
  'is_staff': 1,
  'is_active': 1,
  'date_joined': datetime.datetime(2020, 4, 23, 21, 39, 41, 507829)},
 {'username': 'bill@example.com',
  'password': 'test1234',
  'email': 'bill@example.com',
  'first_name': 'Bill',
  'last_name': 'Jackson',
  'is_superuser': 0,
  'is_staff': 1,
  'is_active': 1,
  'date_joined': datetime.datetime(2020, 4, 23, 21, 39, 41, 507832)},
 {'username': 'sue@example.com',
  'password': 'test1234',
  'email': 'sue@example.com',
  'first_name': 'Sue',
  'last_name': 'Stein',
  'is_superuser': 0,
  'is_staff': 1,
  'is_active': 1,
  'date_jo

In [54]:
# Insert rows to database
session.execute(auth_user_table.insert(), users_import_list)
session.commit()

## Now we can do a SQL query of all users, in Django we would do:
`User.objects.all()`
## Notice I am using the `auth_user_table` that I assigned earlier which is part of SA's `metadata` object

In [13]:
session.query(auth_user_table).all()

[(4,
  'test1234',
  None,
  True,
  'admin@example.com',
  'Admin',
  'admin@example.com',
  True,
  True,
  datetime.datetime(2020, 4, 23, 21, 39, 41, 507769),
  ''),
 (5,
  'test1234',
  None,
  False,
  'chris@example.com',
  'Chris',
  'chris@example.com',
  True,
  True,
  datetime.datetime(2020, 4, 23, 21, 39, 41, 507829),
  'Smith'),
 (6,
  'test1234',
  None,
  False,
  'bill@example.com',
  'Bill',
  'bill@example.com',
  True,
  True,
  datetime.datetime(2020, 4, 23, 21, 39, 41, 507832),
  'Jackson'),
 (7,
  'test1234',
  None,
  False,
  'sue@example.com',
  'Sue',
  'sue@example.com',
  True,
  True,
  datetime.datetime(2020, 4, 23, 21, 39, 41, 507835),
  'Stein'),
 (8,
  'test1234',
  None,
  False,
  'nikhil@example.com',
  'Nikhil',
  'nikhil@example.com',
  True,
  True,
  datetime.datetime(2020, 4, 23, 21, 39, 41, 507847),
  'Anwar')]

## Let's also insert questions to the database

In [19]:
questions_file = os.path.join(CURRENT_DIR, 'questions_data.csv')

with open(questions_file, mode='r') as questions_file:
    reader = csv.reader(questions_file, delimiter=',')
    headers = next(reader, None)
    questions_import_list = [{headers[0]: row[0], "pub_date": dt.datetime.now()
                         } for idx, row in enumerate(reader)]
    

questions_import_list

[{'question_text': "What's new?",
  'pub_date': datetime.datetime(2020, 4, 25, 14, 4, 34, 110321)},
 {'question_text': 'What is your favorite programming language?',
  'pub_date': datetime.datetime(2020, 4, 25, 14, 4, 34, 110329)},
 {'question_text': 'What is your favorite city?',
  'pub_date': datetime.datetime(2020, 4, 25, 14, 4, 34, 110332)},
 {'question_text': 'What is your favorite meetup?',
  'pub_date': datetime.datetime(2020, 4, 25, 14, 4, 34, 110334)},
 {'question_text': 'How are you doing today?',
  'pub_date': datetime.datetime(2020, 4, 25, 14, 4, 34, 110346)}]

In [21]:
# Insert questions to database
session.execute(polls_question_table.insert(), questions_import_list)
session.commit()

## We can query this data similarly to above. In Django, this would be `Question.objects.all()`

In [23]:
session.query(polls_question_table).all()

[(1, "What's new?", datetime.datetime(2020, 4, 25, 14, 4, 34, 110321)),
 (2,
  'What is your favorite programming language?',
  datetime.datetime(2020, 4, 25, 14, 4, 34, 110329)),
 (3,
  'What is your favorite city?',
  datetime.datetime(2020, 4, 25, 14, 4, 34, 110332)),
 (4,
  'What is your favorite meetup?',
  datetime.datetime(2020, 4, 25, 14, 4, 34, 110334)),
 (5,
  'How are you doing today?',
  datetime.datetime(2020, 4, 25, 14, 4, 34, 110346))]

## Let's filter for the question with an id of 1. 
### Django: `Question.objects.filter(id=1)`

In [36]:
query = session.query(polls_question_table).filter(polls_question_table.c.id==1)

for q in query: print(q)

(1, "What's new?", datetime.datetime(2020, 4, 25, 14, 4, 34, 110321))


## Using SQL's  `where` clause with `startswith` in the `polls_question_table`
### Django: `Question.objects.filter(question_text__startswith='What')`

In [40]:
query = session.query(polls_question_table).filter(polls_question_table.c.question_text.startswith('What'))

for q in query: print(q)

(1, "What's new?", datetime.datetime(2020, 4, 25, 14, 4, 34, 110321))
(2, 'What is your favorite programming language?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110329))
(3, 'What is your favorite city?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110332))
(4, 'What is your favorite meetup?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110334))


## You can also use the `like` SQL syntax

In [43]:
query = session.query(polls_question_table).filter(polls_question_table.c.question_text.like('%favorite%'))

for q in query: print(q)

(2, 'What is your favorite programming language?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110329))
(3, 'What is your favorite city?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110332))
(4, 'What is your favorite meetup?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110334))


## Get the question that was published this year.

### Django:

from django.utils import timezone

current_year = timezone.now().year

Question.objects.get(pub_date__year=current_year)

In [11]:
current_year = dt.datetime.now().year # Remember I aliased datetime to dt above

query = session.query(polls_question_table).filter(polls_question_table.c.pub_date >= current_year)

for q in query: print(q)

(1, "What's new?", datetime.datetime(2020, 4, 25, 14, 4, 34, 110321))
(2, 'What is your favorite programming language?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110329))
(3, 'What is your favorite city?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110332))
(4, 'What is your favorite meetup?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110334))
(5, 'How are you doing today?', datetime.datetime(2020, 4, 25, 14, 4, 34, 110346))


## Now, let's do our final SQL import i.e. insert

In [15]:
choices_file = os.path.join(CURRENT_DIR, 'choice_data.csv')

with open(choices_file, mode='r') as choice_file:
    reader = csv.reader(choice_file, delimiter=',')
    headers = next(reader, None)
    choices_import_list = [{headers[0]: row[0], headers[1]: row[1], headers[2]: row[2]
                         } for idx, row in enumerate(reader)]
    

choices_import_list

[{'choice_text': 'Not much', 'votes': '0', 'question_id': '1'},
 {'choice_text': 'Trying to stay safe', 'votes': '0', 'question_id': '1'},
 {'choice_text': 'Bored', 'votes': '0', 'question_id': '1'},
 {'choice_text': 'Doing good', 'votes': '0', 'question_id': '1'},
 {'choice_text': 'Python', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'Java', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'JavaScript', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'Ruby', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'C/C++', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'C#', 'votes': '0', 'question_id': '2'},
 {'choice_text': 'R', 'votes': '0', 'question_id': '2'}]

In [16]:
# Insert choices to database
session.execute(polls_choice_table.insert(), choices_import_list)
session.commit()

# Query the `polls_choice_table` to ensure the data was loaded

In [18]:
session.query(polls_choice_table).all()

[(1, 'Not much', 0, 1),
 (2, 'Trying to stay safe', 0, 1),
 (3, 'Bored', 0, 1),
 (4, 'Doing good', 0, 1),
 (5, 'Python', 0, 2),
 (6, 'Java', 0, 2),
 (7, 'JavaScript', 0, 2),
 (8, 'Ruby', 0, 2),
 (9, 'C/C++', 0, 2),
 (10, 'C#', 0, 2),
 (11, 'R', 0, 2)]

## Let's add one more choice to question 1

### Django

q = Question.objects.get(pk=1)

c = q.choice_set.create(choice_text='Just hacking again', votes=0)

In [21]:
## In SQLAlchemy

In [22]:
session.execute(polls_choice_table.insert(), {'choice_text': 'Attending a Python Meetup', 'votes': '0', 'question_id': '1'})
session.commit()

In [24]:
session.query(polls_choice_table).all()

[(1, 'Not much', 0, 1),
 (2, 'Trying to stay safe', 0, 1),
 (3, 'Bored', 0, 1),
 (4, 'Doing good', 0, 1),
 (5, 'Python', 0, 2),
 (6, 'Java', 0, 2),
 (7, 'JavaScript', 0, 2),
 (8, 'Ruby', 0, 2),
 (9, 'C/C++', 0, 2),
 (10, 'C#', 0, 2),
 (11, 'R', 0, 2),
 (12, 'Attending a Python Meetup', 0, 1)]